In [ ]:
import kaggle
import pandas as pd
import os
import lightgbm as lgbm
import optuna as opt
import seaborn as sns
import numpy as np
import gc
import shap
from scipy.stats import binom_test
from sklearn.model_selection import train_test_split

In [2]:
os.chdir("/Users/matthewlafferty/Dropbox/Kaggle/Jane_Street/jane-street-market-prediction")

In [3]:
data = pd.read_csv('train.csv')

In [4]:
data.sample(10)

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
491892,80,0.059207,0.003374,0.007049,0.010831,-0.012751,-0.014573,1,1.107300,-0.788714,...,-3.344326,2.704979,-2.592726,3.000305,-3.388496,3.263304,-2.804504,3.027151,-2.440436,491892
219626,34,0.285587,0.002744,0.004567,0.007870,0.015364,0.018541,1,0.491193,0.147054,...,-0.090701,1.782374,0.395661,1.424740,0.410524,2.460661,0.678450,2.277390,0.635024,219626
2015110,429,3.146723,-0.000063,-0.001278,-0.002908,-0.004575,-0.002998,1,0.751577,3.397723,...,0.744954,0.234358,2.206868,-0.452832,0.602040,-0.132379,1.528540,0.203499,1.997672,2015110
897312,184,0.566679,0.001359,-0.000410,-0.017521,-0.033944,-0.019181,1,-0.416806,-0.403020,...,-3.112464,-1.288962,-2.222993,-0.995549,-3.221498,-1.500098,-2.159030,-1.432691,-2.029860,897312
1198769,255,2.800629,0.001755,0.008786,0.024165,0.045762,0.033423,1,2.093228,0.089239,...,-0.360148,1.250625,-0.433197,1.347622,-0.055718,1.910187,-0.133741,1.682871,-0.170468,1198769
455783,72,0.334733,0.001044,0.000166,0.002704,-0.014786,-0.019530,1,-3.172026,-3.093182,...,-0.554481,0.870773,-0.790721,0.264029,-2.258830,0.708828,-1.270733,0.877817,-0.874588,455783
1337811,284,0.890416,0.004181,0.009401,0.027605,0.043398,0.030062,-1,1.093788,0.817239,...,0.055214,0.476501,0.331179,0.251048,0.184762,0.229638,-0.167930,0.451875,0.179533,1337811
2311783,486,0.077688,-0.000734,0.000584,0.004784,0.009934,0.007780,-1,-3.172026,-3.093182,...,5.297500,-1.530664,1.852107,-0.688117,5.537706,-1.193737,4.159283,-1.201244,3.337126,2311783
1339810,285,0.000000,-0.002936,-0.002981,0.001015,0.010478,0.009534,-1,2.014357,3.461213,...,NaN,1.874482,-0.048451,1.356119,-0.625356,1.936857,-0.516289,1.817743,-0.378154,1339810
535660,87,1.416155,-0.000310,-0.001555,-0.001050,-0.004225,-0.005818,-1,-2.276139,-1.557572,...,-1.480704,-2.161251,0.272887,-2.299898,-1.777989,-4.166561,-1.227035,-2.936782,-0.312190,535660


# Simple GBM Model

In [5]:
data['target'] = 1 * (data['resp'] >= 0)

In [6]:
features = [x for x in list(data) if 'feature' in x]

In [7]:
def create_shadow_features(X:pd.DataFrame, features:list) -> pd.DataFrame:  
    for feature in features:
        temp = X[feature].to_numpy(copy=True)
        np.random.shuffle(temp) # shuffle the values of each feature to all the features
        X[feature + '_shadow'] = temp
    return X

In [11]:
def params_tune(X:pd.DataFrame, target:str, features:list, num_boost_rounds:int, params:dict, nfolds:int=3, verbose_eval:bool = True, sample:float = 0.5) -> dict:
    
    X = X.sample(frac=sample)
    
    dtrain = lgbm.Dataset(data=X[features],
                          label=X[target],
                          feature_name=features)
    
    model = opt.integration.lightgbm.LightGBMTunerCV(params = params,
                                                     train_set = dtrain,
                                                     num_boost_round = num_boost_rounds,
                                                     nfold = nfolds,
                                                     stratified = True,
                                                     shuffle = True,
                                                     feature_name=features,
                                                     early_stopping_rounds=0.05*num_boost_rounds,
                                                     verbose_eval = verbose_eval,
                                                     seed = 51)
    model.run()
    print(model.best_score)
    return model.best_params

In [13]:
params = {'objective': 'binary',
          'verbosity': -1,
          'boosting_type': 'gbdt',
          'learning_rate': 0.25,
          'is_unbalanced': False,
          'metric': 'auc'}

params_0 = params_tune(X=data,
                       target='target',
                       features=[x for x in features if x not in ['resp','weight']],
                       num_boost_rounds=5000,
                       params=params,
                       nfolds=3,
                       verbose_eval=False,
                       sample = 0.1)

[I 2021-01-10 08:19:50,531] A new study created in memory with name: no-name-9120fd8b-44a7-461c-aa31-6850cc0b32c8


  0%|          | 0/7 [00:00<?, ?it/s]

feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

feature_fraction, val_score: 0.537516:   0%|          | 0/7 [00:41<?, ?it/s]

feature_fraction, val_score: 0.537516:  14%|#4        | 1/7 [00:41<04:07, 41.30s/it][I 2021-01-10 08:20:31,838] Trial 0 finished with value: 0.5375160768507232 and parameters: {'feature_fraction': 0.8}. Best is trial 0 with value: 0.5375160768507232.


feature_fraction, val_score: 0.537516:  14%|#4        | 1/7 [00:41<04:07, 41.30s/it]

feature_fraction, val_score: 0.538813:  14%|#4        | 1/7 [01:45<04:07, 41.30s/it]

feature_fraction, val_score: 0.538813:  29%|##8       | 2/7 [01:45<04:01, 48.32s/it][I 2021-01-10 08:21:36,527] Trial 1 finished with value: 0.5388134106108794 and parameters: {'feature_fraction': 1.0}. Best is trial 1 with value: 0.5388134106108794.


feature_fractio

0.5608243819373179


In [15]:
params_0['metric'] = 'auc'
params_0

{'objective': 'binary',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'learning_rate': 0.25,
 'is_unbalanced': False,
 'metric': 'auc',
 'feature_pre_filter': False,
 'lambda_l1': 4.805704302063685e-07,
 'lambda_l2': 6.497878684050329e-07,
 'num_leaves': 189,
 'feature_fraction': 0.7,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20}

In [18]:
def feature_selection_shap(X:pd.DataFrame, features:list, target:str, params:dict, num_boost_rounds:int=10000, sample:float = 0.7, num_iter:int = 3) -> list:
    
    # We will determine optimal parameters on a random sample. We'll use these parameters to construct a new model for each iteration.
    # Determining the hyperparameters takes time, so we'll do it once, and use these parameters to construct our models.
    X_shadow = create_shadow_features(X, features)
    
    # These are the features, categorical features, and categorical feature indices that we will use once the shadow features are added.
    shadow_features = features + [x + '_shadow' for x in features]
                        
    # This is the data frame that we will use to store our SHAP value information
    results_df = pd.DataFrame(columns = shadow_features + [target])
    
    # With the parameters determined, we'll build models using the parameters on a random sample of records, determine the SHAP values on
    # the remaining records.
    for i in range(num_iter):
        X_train, X_shap, y_train, y_shap = train_test_split(X_shadow[shadow_features], X_shadow[target], train_size=sample, shuffle=True, stratify=X_shadow[target])
        
        dtrain = lgbm.Dataset(data = X_train,
                              label = y_train,
                              feature_name = shadow_features)
        
        dval = lgbm.Dataset(data = X_shap,
                            label = y_shap,
                            feature_name = shadow_features)
        
        del(X_train, y_train)
        gc.collect()
        
        current_model = lgbm.train(params=params,
                                   train_set=dtrain,
                                   num_boost_round=num_boost_rounds,
                                   valid_sets=dval,
                                   feature_name=shadow_features,
                                   early_stopping_rounds=int(0.05*num_boost_rounds),
                                   verbose_eval=100)
        
        shap_values = shap.TreeExplainer(current_model).shap_values(X_shap)
        shap_values_df = pd.DataFrame(data = shap_values[1], columns = shadow_features)
        shap_values_df[target] = np.where(y_shap == 1, 1, 0)
        
        print('Missing targets in y_shap = ', sum(y_shap.isna()))
        print('Missing targets in y_shap = ', sum(y_shap))
        print('Missing targets in shap_values_df = ', sum(shap_values_df['target'].isna()))
        print('Missing targets in shap_values_df = ', sum(shap_values_df['target']))
        
        #print(results.shape)
        #print(np.abs(shap_values[0]).mean(0))
        print(results_df.shape)
        #print(results_df.sample(10))
        print(shap_values_df.shape)
        print([x for x in list(results_df) if x not in list(shap_values_df)])
        print([x for x in list(shap_values_df) if x not in list(results_df)])
        #print(shap_values_df.sample(10))
        results_df = pd.concat([results_df, shap_values_df], axis = 0)
    
    cols_to_keep = []
    n = results_df.shape[0]
    for col in features:
        x = np.sum(np.where((results_df[target] == 0) & (results_df[col] < results_df[col + '_shadow']), 1,
                        np.where((results_df[target] == 1) & (results_df[col] > results_df[col + '_shadow']), 1, 0)))
        p_val = binom_test(x=x, n=n, p=0.5, alternative='greater')
        if p_val <= 0.05:
            cols_to_keep += [col]
            
    return cols_to_keep

In [ ]:
features_to_keep = feature_selection_shap(X=data,
                                          features=features,
                                          target='target',
                                          params=params_0,
                                          num_boost_rounds=20000,
                                          sample=0.7,
                                          num_iter=3)

Training until validation scores don't improve for 1000 rounds
[100]	valid_0's auc: 0.615545
[200]	valid_0's auc: 0.620539
[300]	valid_0's auc: 0.622757
[400]	valid_0's auc: 0.621391
[500]	valid_0's auc: 0.62185
[600]	valid_0's auc: 0.622995
[700]	valid_0's auc: 0.624099
[800]	valid_0's auc: 0.622831
[900]	valid_0's auc: 0.625327
[1000]	valid_0's auc: 0.624036
[1100]	valid_0's auc: 0.625355
[1200]	valid_0's auc: 0.626502
[1300]	valid_0's auc: 0.626611
[1400]	valid_0's auc: 0.626031
[1500]	valid_0's auc: 0.626124
[1600]	valid_0's auc: 0.625573
[1700]	valid_0's auc: 0.625923
[1800]	valid_0's auc: 0.625734
[1900]	valid_0's auc: 0.626323
[2000]	valid_0's auc: 0.6267
[2100]	valid_0's auc: 0.627149
[2200]	valid_0's auc: 0.626988
[2300]	valid_0's auc: 0.626752
[2400]	valid_0's auc: 0.626532
[2500]	valid_0's auc: 0.625905
[2600]	valid_0's auc: 0.626303
[2700]	valid_0's auc: 0.626939
[2800]	valid_0's auc: 0.62694
[2900]	valid_0's auc: 0.626564
[3000]	valid_0's auc: 0.627321
Early stopping, best

In [51]:
test_list

['resp']

In [ ]:
dtrain = lgbm.Dataset(data=data[[x for x in features if x not in ['resp','weight']]],
                          label=data['target'],
                          feature_name=[x for x in features if x not in ['resp','weight']])

test_params['metric'] = 'auc'
test_model = lgbm.cv(params = test_params,
                        train_set = dtrain,
                        num_boost_round=10000,
                        nfold=5,
                        stratified=True,
                        shuffle=True,
                        metrics='auc',
                        feature_name=[x for x in features if x not in ['resp','weight']],
                        early_stopping_rounds=100,
                        verbose_eval = 10)

In [ ]:
def num_trees_tune(X:pd.DataFrame, target:str, features:list, num_boost_rounds:int, params:dict, learning_rate:float = 0.01, num_iter:int = 10, train_sample:float = 0.7, verbose_eval:int = 0):
    
    cat_features_indices = [i for i in range(len(features)) if features[i] in cat_features]
    params['learning_rate'] = learning_rate
    best_iter_list = []
        
    for i in range(num_iter):
        X_train, X_val, y_train, y_val = train_test_split(X[features], X[target], train_size = train_sample, shuffle = True, stratify = X[target])
        if len(cat_features) == 0:
            X_train_smote, y_train_smote = SMOTE(random_state=51).fit_resample(X_train, y_train)
        else:
            X_train_smote, y_train_smote = SMOTENC(categorical_features=cat_features_indices, random_state=51).fit_resample(X_train, y_train)
        
        del(X_train, y_train)
        gc.collect()
        
        dtrain = lgbm.Dataset(data = X_train_smote,
                                label = y_train_smote,
                                feature_name = features,
                                categorical_feature = cat_features)
        
        dval = lgbm.Dataset(data = X_val,
                            label = y_val,
                            feature_name = features,
                            categorical_feature = cat_features)
        
        del(X_train_smote, y_train_smote, X_val, y_val)
        gc.collect()
        
        current_model = lgbm.train(params=params,
                                    train_set=dtrain,
                                    num_boost_round=num_boost_rounds,
                                    valid_sets=[dval],
                                    feature_name=features,
                                    categorical_feature=cat_features,
                                    early_stopping_rounds=1000,
                                    verbose_eval=0)

        best_iter_current = current_model.best_iteration
        best_iter_list += [best_iter_current]
        
        print(current_model.best_score)

    return int(np.nanmean(best_iter_list))

In [ ]:
results_df = pd.DataFrame(columns = ['target', 'param', 'val'])

params = {'objective': 'binary',
            'metric': 'binary_logloss',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'learning_rate': 0.25,
            'is_unbalanced': False}
    
for target in targets:
    train = train_all.copy()
    augmented_vals = augmenting_vals(train, target, num = 20)
    train = pd.concat([train, augmented_vals], axis = 0, ignore_index = True)
    best_params = params_tune(X=train, target=target, features=features, cat_features=cat_features, num_boost_rounds=10000, params=params)
    
    features_to_keep = feature_selection_shap(X=train[features], y=train[target], params=best_params, features=features, cat_features=cat_features, sample=0.5, num_boost_rounds=10000, num_iter = 3, verbose_eval = 0)
    cat_features_to_keep = [x for x in cat_features if x in features_to_keep]
    
    best_params['features'] = features_to_keep
    best_params['cat_features'] = cat_features_to_keep
    
    num_trees = num_trees_tune(X=train, target=target, features=features_to_keep, cat_features=cat_features_to_keep, num_boost_rounds=50000, params=best_params, learning_rate = 0.01, num_iter = 10, train_sample = 0.7, verbose_eval = 0)
    best_params['num_trees'] = int(1.1*num_trees)
    
    for key in best_params.keys():
        results_df = pd.concat([results_df, pd.DataFrame(data=[[target, key, best_params[key]]], columns=['target', 'param', 'val'])], axis=0, ignore_index=True)
    print(results_df.tail(10))
    print(target, ' completed. ', len(targets) - targets.index(target) - 1, ' to go...')